In [18]:
### pry-docの読み込み
#require "/root/git_jupyter_notebook/Ruby/vendor/bundle/ruby/2.3.0/gems/pry-doc-0.10.0/lib/pry-doc"
require "/Users/ftakao2007/jupyter/vendor/bundle/ruby/2.3.0/gems/pry-doc-0.10.0/lib/pry-doc"
### エンコーディングをUTF-8に設定
Encoding.default_external = "UTF-8"

"UTF-8"

## クラスオブジェクト

* クラスレイヤとインスタンスレイヤ
    * クラスレイヤ
        * 例
            * クラス(Fooなど)
            * クラスオブジェクト(インスタンスメソッド method1など)
    * インスタンスレイヤ
        * クラスからオブジェクトを生成(インスタンス化)するとインスタンスレイヤに降りてくるというイメージ
        * 例
            * Fooクラスのインスタンス(foo1, foo2など)
                * それぞれのインスタンスのインスタンス変数(@aなど)

* 要素
    * インスタンスメソッド
        * メモリ上では<font color="red">クラスオブジェクトに保持</font>される
            * レイヤとしてはクラスレイヤ
    * インスタンス変数
        * メモリ上では<font color="red">インスタンスに保持</font>される
            * レイヤとしてはインスタンスレイヤ
                

## 継承したクラスオブジェクト

* Foo : 継承されるクラスオブジェクト
    * 継承するクラスより一般的な性質を持つ(汎化)
* FooExt : 継承したクラスオブジェクト
    * 元のクラスより固有の性質を持つ(特化)

* foo1
    * Fooクラスで定義された性質を持つ
* fooExt1
    * Fooクラス及びFooExtクラスで定義された性質を持つ

In [1]:
class Foo
  def initialize(a)
    @a = a
  end
  
  def method1
    @a
  end
end

### FooExt <-(特化)--(汎化)-> Foo(継承元クラス)
class FooExt < Foo
  def initialize(a,b)
    @b = b
    super a
  end
  
  def method2(c)
    @a + @b + c
  end
end

### Fooクラスのインスタンスを生成
foo1 = Foo.new(1)
foo2 = Foo.new(2)

### 定義したインスタンスメソッドを実行
puts foo1.method1
puts foo2.method1

### FooExtクラスのインスタンスを生成
fooExt1 = FooExt.new(3, 4)

### 定義したインスタンスメソッドを実行
puts fooExt1.method1
puts fooExt1.method2(5)  

### スーパークラスの取得
### FooExt -(superclass)-> Foo
puts FooExt.superclass

1
2
3
12
Foo


## メソッドの探索経路

### メソッドが存在する場合

1. foo1インスタンスのインスタンスメソッドmethod1を実行
1. インタプリタがそれが属するクラス(Foo)のオブジェクトに指定されたメソッド(method1)があるかどうかを判定
    * 抽象度を一つ上げて(クラスレイヤで)Fooのクラスオブジェクトにmethod1が存在するかを判定している
1. 同様にfooExt1インスタンスのmethod2メソッドが実行された場合、クラスレイヤのFooExtクラスにmethod2が存在するかを判定
1. メソッドがある場合は正常に実行される

### メソッドが存在しない場合

1.  インスタンスメソッド実行
1. そのクラスのクラスオブジェクト探す
1. スーパークラスのクラスオブジェクトを順にたどって探す
1. 最後まで見つからなければ例外NoMethodErrorを発生

In [2]:
class Foo
  def initialize(a)
    @a = a
  end
  
  def method1
    @a
  end
end

class FooExt < Foo
  def initialize(a,b)
    @b = b
    super a
  end
  
  def method2(c)
    @a + @b + c
  end
end

### fooExt1にmethod1は存在しないが、継承元にはあるのでそちらが実行される
fooExt1 = FooExt.new(1,2)
puts fooExt1.method1

### Fooにmethod2は存在しないのでNoNameError
foo1 = Foo.new(1)
begin
  puts foo1.method2
rescue NoMethodError => ex
  puts ex
end

1
undefined method `method2' for #<Foo:0x007fa3f1966e90 @a=1>


### method_missing

* メソッドが見つからないとき
    * Objectクラスのmethod_misssingメソッド(クラスメソッド?)が呼び出される(NoMethodError)
* method_misssingメソッド
    * デフォルト
        * 第一引数 : 指定されたメソッド名
        * 第二引数以降 : 指定された引数
* method_missingを上書きすることでメソッドが見つからない場合の動作をフックすることができる
* 参考
    * OSS等でどこにもメソッドが定義されていない場合
        1. method_missingでフックされている
        2. 動的にメソッドを生成している

In [15]:
class Fuga
  def method_missing(m, *args)
    puts "called: " + m.to_s
    puts args
    puts 

    ### superで例外が発生するようにスーパークラスのmethod_missingを呼び出す
    begin
      super
    rescue NoMethodError => ex
      puts ex
    end
    
  end
end

Fuga.new.no_method(1, nil, true, :a, "hoge")

called: no_method
[1, nil, true, :a, "hoge"]

undefined method `no_method' for #<Fuga:0x007fa3f13b58c0>


## 継承チェーン(継承順)

* スーパークラスを省略してクラスを定義
    * <font color="red">Objectクラスが継承される</font>
        * Ruby1.9以降ではObjectクラスはBasicObjectクラスを継承するように変更されている
            * BasicObjectクラスはRuby1.9より前のObjectクラスからほとんどのメソッドを取り除いたクラス
            * プログラム作成上ではBasicObjectクラスを意識する必要はないが試験では継承チェーンの問題で出題される事がある
        * ObjectクラスとBasicObjectクラスの間にKernelモジュールがある
        * FooクラスはObjectクラスを継承している
* クラス継承チェーンの参照
    * クラス継承チェーン一覧
        * ancestorsメソッド
    * あるクラスが継承チェーンンに含まれているか確認
        * 比較演算子

In [16]:
### スーパークラスを略さずクラスを定義
class Foo < Object
  def initialize(a)
    @a = a
  end
  
  def method1
    @a
  end
end

class FooExt < Foo
  def initialize(a,b)
    @b = b
    super a
  end
  
  def method2(c)
    @a + @b + c
  end
end

### クラス継承チェーン一覧の表示
### Jupyter環境なので途中でPP::ObjectMixin, JSON::Ext::Generator::GeneratorMethods::Objectが表示されている
puts Foo.ancestors
puts FooExt.ancestors

### 比較演算子でクラスが継承チェーンに含まれているかを確認
### 不等号がクラスの大小関係(包含関係)を表していると覚える
puts Foo < Object
puts Foo < FooExt
puts Foo > FooExt

[Foo, Object, PP::ObjectMixin, JSON::Ext::Generator::GeneratorMethods::Object, Kernel, BasicObject]
[FooExt, Foo, Object, PP::ObjectMixin, JSON::Ext::Generator::GeneratorMethods::Object, Kernel, BasicObject]
true
false
true


## オブジェクトが持つインスタンスメソッドとインスタンス変数の調査

* インスタンスメソッドの調査
    * instance_methodsメソッド
        * 引数にfalseを指定するとスーパークラスをたどらない
* インスタンス変数の調査
    * instance_valuesメソッド
* 参考
    * insance_valuesメソッドはKernelモジュールのインスタンスメソッド
    * instance_methodsメソッドはクラスメソッド？

In [23]:
Module.instance_methods(false).grep(/instance_methods/)

[:public_instance_methods, :instance_methods, :private_instance_methods, :protected_instance_methods]

In [24]:
class Foo
  def initialize(a)
    @a = a
  end
  
  def method1
    @a
  end
end

class FooExt < Foo
  def initialize(a,b)
    @b = b
    super a
  end
  
  def method2(c)
    @a + @b + c
  end
end

### インスタンスメソッド
puts Foo.instance_methods(false)
puts FooExt.instance_methods(false)

### インスタンス変数
foo1 = Foo.new(1)
fooExt1 = FooExt.new(1,2)
puts foo1.instance_variables
puts fooExt1.instance_variables

puts "引数にfalseを指定しない場合は継承元クラスのインスタンスメソッドが全て表示される"
puts FooExt.instance_methods
puts

puts "BasicObjectのインスタンスメソッド"
puts BasicObject.instance_methods(false)
puts

puts "Kernelモジュールのインスタンスメソッド(ここにinstance_variablesがある)"
puts Kernel.instance_methods(false)
puts

puts "Objectのインスタンスメソッド"
puts Object.instance_methods(false)
puts

puts "instance_methodsはクラスメソッドなのでこれらの中には含まれていない？"

[:method1]
[:method2]
[:@a]
[:@b, :@a]
引数にfalseを指定しない場合は継承元クラスのインスタンスメソッドが全て表示される
[:method2, :method1, :pry, :__binding__, :pretty_print, :pretty_print_cycle, :pretty_print_instance_variables, :pretty_print_inspect, :to_json, :instance_of?, :public_send, :instance_variable_get, :instance_variable_set, :instance_variable_defined?, :remove_instance_variable, :private_methods, :kind_of?, :instance_variables, :tap, :define_singleton_method, :is_a?, :extend, :to_enum, :enum_for, :<=>, :pretty_inspect, :===, :=~, :!~, :eql?, :respond_to?, :freeze, :inspect, :display, :object_id, :send, :gem, :to_s, :method, :public_method, :singleton_method, :nil?, :hash, :class, :singleton_class, :clone, :dup, :itself, :taint, :tainted?, :untaint, :untrust, :trust, :untrusted?, :methods, :protected_methods, :frozen?, :public_methods, :singleton_methods, :!, :==, :!=, :__send__, :equal?, :instance_eval, :instance_exec, :__id__]

BasicObjectのインスタンスメソッド
[:!, :==, :!=, :__send__, :equal?, :instance_eval, :insta

## メソッドに別名をつける

* 別名をつける
    * alias <新メソッド名> <旧メソッド名>
* 定義を取り消す(別名を消す)
    * undef <メソッド名>
* 挙動
    * 評価された時点でのメソッド定義が対象
        * 定義される前にaliasやundefを書いてもNameErrorが発生する

In [5]:
class Hoge
  def fuga1; end
  def fuga2; end
  alias fuga3 :fuga1
  undef fuga2
end

puts Hoge.instance_methods(false)

[:fuga1, :fuga3]


## オープンクラス

* オープンクラス
    * 定義済みのクラスを再定義できるクラスのこと
* クラスの再オープン
    * 一度定義したクラスを再定義のために開く事
* 既存の標準クラスも再定義できる
    * Stringなど
    * 標準クラスはプログラム全体で有効となるので注意が必要

In [29]:
### Hogeクラス定義1回目
class Hoge
  def fuga4; end
end

### Hogeクラス定義2回目
class Hoge
  def fuga1; end
  def fuga2; end
  alias fuga3 :fuga1
  undef fuga2
end

puts "Hogeクラス定義1回目のメソッドfuga4が存在する"
puts Hoge.instance_methods(false)
puts

class String
  def fuga5; end
end

puts "既存のStringクラスにfuga5メソッドを追加"
puts String.instance_methods(false).grep(/f/)

Hogeクラス定義1回目のメソッドfuga4が存在する
[:fuga4, :fuga1, :fuga3]

既存のStringクラスにfuga5メソッドを追加
[:fuga5, :freeze, :force_encoding, :to_f]


## スーパークラスを指定して再オープン

* スーパークラスを指定して再オープン
    * スーパークラスはオープンする前のクラスと同じでなければならない

In [30]:
### クラスFoo, BarとFooから継承したクラスBazを定義
class Foo; end
class Bar; end
class Baz < Foo
end

### BarはBazのスーパークラスではないので再オープンできない
#class Baz < Bar
#end

### FooはBazのスーパークラスなので再オープンできる
class Baz < Foo
end

### Bazは再オープンできる
class Baz
end